In [1]:
from bs4 import BeautifulSoup

In [2]:
import numpy as np

In [3]:
import requests

In [7]:
import pandas as pd

In [9]:
def get_title(soup):

    try:  
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [10]:
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

In [11]:
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

In [12]:
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count


In [13]:
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [18]:
if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5'
    }
    URL = "https://www.amazon.com/s?k=shoes&crid=5VF8WKI47K80&sprefix=shoes%2Caps%2C343&ref=nb_sb_noss_1"
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})
    links_list = [link.get('href') for link in links]
    
    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}
    
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df.replace({'title': {'': np.nan}}, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [19]:
amazon_df

,title,price,rating,reviews,availability
0,Amazon Essentials Women's Pointed-Toe Ballet Flat,,4.2 out of 5 stars,"10,494 ratings",
1,Under Armour Men's Charged Assert 9 Running Shoe,,4.6 out of 5 stars,"59,399 ratings",Not Available
2,adidas Men's Daily 3.0 Sneaker,,4.7 out of 5 stars,"15,720 ratings",Not Available
3,Hey Dude Mens Wally Sox,,4.7 out of 5 stars,"132,853 ratings",Not Available
4,New Balance Men's 608 V5 Casual Comfort Cross ...,,4.6 out of 5 stars,"77,522 ratings",Not Available
...,...,...,...,...,...
62,Arkbird Men's Leather Sandals Athletic Fisherm...,,4.6 out of 5 stars,136 ratings,In Stock
63,"Men's House Slippers, Cozy Non-slip Home Shoes...",,4.4 out of 5 stars,161 ratings,In Stock
64,"Professional Shoe Lift for Uneven Legs, Shoe b...",,3.5 out of 5 stars,97 ratings,In Stock
65,"ELITE Men's Peak Bowling Shoes - 40% Lighter, ...",,4.2 out of 5 stars,38 ratings,
